In [236]:
import mlflow as mf
from pathlib import Path
import itertools
import pandas as pd
import numpy as np

In [237]:
mf.set_tracking_uri(str(Path('../code/train/mlruns').absolute()))

In [238]:
algs = ['lgb', 'svr','rf','mlp','ada','resnet1d','spectroresnet','mlpbp','unet1d','ppgiabp','vnet']
tars = ['SP','DP']
mets = ['mae', 'me', 'std', 'mase']
mets2 = [i+'_'+j for i, j in itertools.product(tars, mets)]
dbs = ['sensors', 'uci2', 'bcg', 'ppgbp']

In [239]:
exp = mf.get_experiment_by_name(f'sensors-resnet1d')
runs = mf.search_runs([exp.experiment_id])
runs = runs[runs['tags.mlflow.parentRunId'].isnull()].reset_index(drop=True)
runs = runs[runs.status=='FINISHED'].reset_index(drop=True)
runs = runs[runs['tags.mlflow.source.name']=='train.py'].reset_index(drop=True)

In [247]:
runs['metrics.test/sbp_mae']

0    17.459
1    25.628
2    23.694
Name: metrics.test/sbp_mae, dtype: float64

In [244]:
runs[[c for c in runs.columns if 'param' in c]].iloc[0]

params.epochs                                                                    None
params.weight_decay                                                              None
params.amsgrad                                                                   None
params.stopped_epoch                                                             None
params.eps                                                                       None
params.mode                                                                       min
params.lr                                                                       0.001
params.monitor                                                                val_mse
params.betas                                                                     None
params.patience                                                                    10
params.optimizer_name                                                            None
params.min_delta                                      

In [138]:
l_df= []
for db in dbs:
    df_metrics = pd.DataFrame(index=algs,columns=mets2)
    for alg in algs:
        if not(alg in ['resnet1d','spectroresnet','mlpbp','unet1d','ppgiabp','vnet']):
            for tar in tars:
                print(db, alg, tar)
                exp = mf.get_experiment_by_name(f'{db}-{alg}-{tar}')
                if not(exp is None):
                    runs = mf.search_runs([exp.experiment_id])
                    runs = runs[runs['tags.mlflow.parentRunId'].isnull()].reset_index(drop=True)
                    runs = runs[runs.status=='FINISHED'].reset_index(drop=True)
                    runs = runs[runs['tags.mlflow.source.name']=='train.py'].reset_index(drop=True)
                    if runs.shape[0] > 0:
                        cols1 = [tar+'_'+m for m in mets if m!='mase']
                        cols2 = ['metrics.ts/'+m for m in cols1]
                        df_metrics.loc[alg,cols1] = runs.iloc[0][cols2].values
                        df_metrics.loc[alg,f'{tar}_mase'] = runs.iloc[0][f'metrics.ts/{tar}_mae']/runs.iloc[0][f'metrics.nv/{tar}_mae'] *100
        else: 
            print(db, alg)
            exp = mf.get_experiment_by_name(f'{db}-{alg}')
            if not(exp is None):
                runs = mf.search_runs([exp.experiment_id])
                runs = runs[runs['tags.mlflow.parentRunId'].isnull()].reset_index(drop=True)
                runs = runs[runs.status=='FINISHED'].reset_index(drop=True)
                runs = runs[runs['tags.mlflow.source.name']=='train.py'].reset_index(drop=True)
                if runs.shape[0] > 0:
                    cols1 = [i+'_'+j for i, j in itertools.product(tars, mets) if j != 'mase']
                    cols2 = ['metrics.test/'+m.replace('SP','sbp').replace('DP','dbp') for m in cols1]
                    
                    df_metrics.loc[alg,cols1] = runs.iloc[0][cols2].values
                    df_metrics.loc[alg,f'SP_mase'] = runs.iloc[0][f'metrics.test/sbp_mae']/runs.iloc[0][f'metrics.nv/sbp_mae'] *100
                    df_metrics.loc[alg,f'DP_mase'] = runs.iloc[0][f'metrics.test/dbp_mae']/runs.iloc[0][f'metrics.nv/dbp_mae'] *100
                    
    l_df.append(df_metrics)
df_metrics = pd.concat(l_df)

sensors lgb SP
sensors lgb DP
sensors svr SP
sensors svr DP
sensors rf SP
sensors rf DP
sensors mlp SP
sensors mlp DP
sensors ada SP
sensors ada DP
sensors resnet1d
sensors spectroresnet
sensors mlpbp
sensors unet1d
sensors ppgiabp
sensors vnet
uci2 lgb SP
uci2 lgb DP
uci2 svr SP
uci2 svr DP
uci2 rf SP
uci2 rf DP
uci2 mlp SP
uci2 mlp DP
uci2 ada SP
uci2 ada DP
uci2 resnet1d
uci2 spectroresnet
uci2 mlpbp
uci2 unet1d
uci2 ppgiabp
uci2 vnet
bcg lgb SP
bcg lgb DP
bcg svr SP
bcg svr DP
bcg rf SP
bcg rf DP
bcg mlp SP
bcg mlp DP
bcg ada SP
bcg ada DP
bcg resnet1d
bcg spectroresnet
bcg mlpbp
bcg unet1d
bcg ppgiabp
bcg vnet
ppgbp lgb SP
ppgbp lgb DP
ppgbp svr SP
ppgbp svr DP
ppgbp rf SP
ppgbp rf DP
ppgbp mlp SP
ppgbp mlp DP
ppgbp ada SP
ppgbp ada DP
ppgbp resnet1d
ppgbp spectroresnet
ppgbp mlpbp
ppgbp unet1d
ppgbp ppgiabp
ppgbp vnet


In [6]:
df_metrics

,SP_mae,SP_me,SP_std,SP_mase,DP_mae,DP_me,DP_std,DP_mase
lgb,15.63,-0.047,19.635,88.761429,7.608,-0.017,9.818,92.039681
svr,15.605,-0.005,19.678,88.619456,7.502,-1.451,9.812,90.757319
rf,15.863,-0.117,19.848,90.084616,7.657,-0.034,9.864,92.63247
mlp,16.029,-0.498,20.1,91.027316,7.774,-0.194,10.039,94.047907
ada,15.752,-0.062,19.773,89.454256,7.68,-0.271,9.961,92.910719
resnet1d,17.459,-0.12,21.703,99.148163,8.329,-2.507,10.781,100.762158
spectroresnet,17.834,0.901,28.053,101.277756,8.309,0.127,11.076,100.520203
mlpbp,17.615,0.014,21.86,100.034073,8.258,-0.021,10.509,99.903218
unet1d,15.64,-1.156,19.641,88.818218,7.656,-0.448,9.929,92.620373
ppgiabp,16.446,-3.233,20.411,93.395423,7.988,-0.31,10.28,96.636826


### Table for paper

In [26]:
algs_names = {'lgb':'LightGBM',
                'svr':'SVR',
                'rf':'RF',
                'mlp':'MLP',
                'ada':'AdaBoost',
                'resnet1d':'ResNet',
                'spectroresnet':'SpectroResNet',
                'mlpbp':'MLPBP',
                'unet1d':'UNet',
                'ppgiabp':'PPGIABP',
              'vnet':'V-Net'}

In [139]:
df_lat = df_metrics.copy().astype(float)
df_lat = df_lat.reset_index()
df_lat = df_lat.rename(columns={'index':'alg'})
df_lat['alg'] = df_lat.alg.map(lambda a: algs_names[a])
df_lat['SP_mestd'] = df_lat['SP_me'].abs()+df_lat['SP_std']
df_lat['DP_mestd'] = df_lat['DP_me'].abs()+df_lat['DP_std']
df_lat = df_lat.round(2)

In [140]:
n_alg = len(algs_names.keys())
for i in range(4):
    df_temp=df_lat.iloc[i*n_alg:(i+1)*n_alg]
    best = [df_temp[c].argmin() for c in ['SP_mae','SP_mestd','DP_mae','DP_mestd']]
    df_lat_temp = pd.DataFrame()
    df_lat_temp['alg'] = df_temp['alg'].values
    for tar in ['SP','DP']:
        df_lat_temp[tar+'_MAE'] = df_temp[tar+'_mae'].map(lambda n: '{:.2f}'.format(n)).values
        df_lat_temp[tar+'_MESTD'] = (df_temp[tar+'_me'].map(lambda n: '{:.2f}'.format(n)) + '$\pm$' + df_temp[tar+'_std'].map(lambda n: '{:.2f}'.format(n))).values
        df_lat_temp[tar+'_MASE'] = df_temp[tar+'_mase'].map(lambda n: '{:.2f}'.format(n)).values
        
        
    df_lat_temp.loc[best[0],'SP_MAE'] = '\textbf{'+df_lat_temp.loc[best[0],'SP_MAE']+'}'
    df_lat_temp.loc[best[0],'SP_MASE'] = '\textbf{'+df_lat_temp.loc[best[0],'SP_MASE']+'}'
    df_lat_temp.loc[best[1],'SP_MESTD'] = '\textbf{'+df_lat_temp.loc[best[1],'SP_MESTD']+'}'
    
    df_lat_temp.loc[best[2],'DP_MAE'] = '\textbf{'+df_lat_temp.loc[best[2],'DP_MAE']+'}'
    df_lat_temp.loc[best[2],'DP_MASE'] = '\textbf{'+df_lat_temp.loc[best[2],'DP_MASE']+'}'
    df_lat_temp.loc[best[3],'DP_MESTD'] = '\textbf{'+df_lat_temp.loc[best[3],'DP_MESTD']+'}'
    print(df_lat_temp.set_index('alg').to_latex().replace("textbackslash ","").replace("\$","$").replace("\{","{").replace("\}","}"))

\begin{tabular}{lllllll}
\toprule
{} &          SP\_MAE &                  SP\_MESTD &         SP\_MASE &         DP\_MAE &                 DP\_MESTD &         DP\_MASE \\
alg           &                 &                           &                 &                &                          &                 \\
\midrule
LightGBM      &           15.63 &  \textbf{-0.05$\pm$19.64} &           88.76 &           7.61 &  \textbf{-0.02$\pm$9.82} &           92.04 \\
SVR           &  \textbf{15.60} &           -0.00$\pm$19.68 &  \textbf{88.62} &  \textbf{7.50} &           -1.45$\pm$9.81 &  \textbf{90.76} \\
RF            &           15.86 &           -0.12$\pm$19.85 &           90.08 &           7.66 &           -0.03$\pm$9.86 &           92.63 \\
MLP           &           16.03 &           -0.50$\pm$20.10 &           91.03 &           7.77 &          -0.19$\pm$10.04 &           94.05 \\
AdaBoost      &           15.75 &           -0.06$\pm$19.77 &           89.45 &           7.68 &        

/tmp/ipykernel_2519774/1409821673.py:20: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_lat_temp.set_index('alg').to_latex().replace("textbackslash ","").replace("\$","$").replace("\{","{").replace("\}","}"))
/tmp/ipykernel_2519774/1409821673.py:20: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_lat_temp.set_index('alg').to_latex().replace("textbackslash ","").replace("\$","$").replace("\{","{").replace("\}","}"))
/tmp/ipykernel_2519774/1409821673.py:20: Fut

### Table MASE results

In [175]:
algs_names = {'lgb':'LightGBM',
                'svr':'SVR',
                'rf':'RF',
                'mlp':'MLP',
                'ada':'AdaBoost',
                'resnet1d':'ResNet',
                'spectroresnet':'SpectroResNet',
                'mlpbp':'MLPBP',
                'unet1d':'UNet',
                'ppgiabp':'PPGIABP',
              'vnet':'V-Net'}
db_names = {'sensors':'Sensors', 'uci2':'UCI', 'bcg':'BCG', 'ppgbp':'PPGBP'}
f2l = ['lgb', 'svr','rf','mlp','ada']
s2l = ['resnet1d','spectroresnet','mlpbp']
s2s = ['unet1d','ppgiabp','vnet']
n_alg = len(algs_names.keys())

In [231]:
best_f2l = []
best_s2l = []
best_s2s = []
best = []
df_mase = pd.DataFrame()
for tar in ['SP','DP']:
    for i,db in enumerate(dbs):
        df_mase[db+'_'+tar] = df_metrics.iloc[i*n_alg:(i+1)*n_alg][tar+'_mase'].astype(np.float64)
        best_f2l.append(df_mase.loc[f2l,db+'_'+tar].idxmin())
        best_s2l.append(df_mase.loc[s2l, db+'_'+tar].idxmin())
        best_s2s.append(df_mase.loc[s2s,db+'_'+tar].idxmin())
        best.append(df_mase[db+'_'+tar].idxmin())
df_mase = df_mase.round(2).fillna(-1)
df_mase = df_mase.applymap(lambda v: '-' if v ==-1 else str(v))
for i, col in enumerate(df_mase.columns):
    df_mase.loc[best[i], col] = '\textbf{'+df_mase.loc[best[i], col]+'}'
    df_mase.loc[best_f2l[i], col] = df_mase.loc[best_f2l[i], col]+"$\star$"
    df_mase.loc[best_s2l[i], col] = df_mase.loc[best_s2l[i], col]+"$\star$"
    if not('ppgbp'in col):
        df_mase.loc[best_s2s[i], col] = df_mase.loc[best_s2s[i], col]+"$\star$"
df_mase.index = df_mase.index.map(lambda a: algs_names[a])      

In [235]:
print(df_mase.to_latex().replace("textbackslash ","").replace("\$","$").replace("\{","{").replace("\}","}"))

\begin{tabular}{lllllllll}
\toprule
{} &             sensors\_SP &                uci2\_SP &                 bcg\_SP &               ppgbp\_SP &             sensors\_DP &                uci2\_DP &                 bcg\_DP &               ppgbp\_DP \\
\midrule
LightGBM      &                  88.76 &            95.6$\star$ &                   98.8 &  \textbf{79.76}$\star$ &                  92.04 &                  96.07 &                  99.19 &                  92.18 \\
SVR           &  \textbf{88.62}$\star$ &                  99.02 &                  93.07 &                  80.29 &  \textbf{90.76}$\star$ &           94.46$\star$ &                  92.75 &                   90.9 \\
RF            &                  90.08 &                   95.6 &                 104.72 &                  80.42 &                  92.63 &                  96.48 &                  99.77 &                  91.76 \\
MLP           &                  91.03 &                 103.18 &                  105.5 &

/tmp/ipykernel_2519774/2840221613.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_mase.to_latex().replace("textbackslash ","").replace("\$","$").replace("\{","{").replace("\}","}"))


### Table for testing 

In [41]:
exp = mf.get_experiment_by_name('uci2-spectroresnet')
runs = mf.search_runs([exp.experiment_id])
runs = runs[runs['tags.mlflow.parentRunId'].isnull()].reset_index(drop=True)
runs = runs[runs.status=='FINISHED'].reset_index(drop=True)
runs = runs[runs['tags.mlflow.source.name']=='train.py'].reset_index(drop=True)

In [42]:
runs2 = mf.search_runs([exp.experiment_id])
runs2[runs2['tags.mlflow.parentRunId']==runs.iloc[0]['run_id']][['metrics.test/sbp_mae','metrics.test/dbp_mae']]

,metrics.test/sbp_mae,metrics.test/dbp_mae
0,19.876,9.003


In [22]:
runs.iloc[0]['run_id']

'cc766d24cd3b45f5a8c1b36bbea91d13'

In [40]:
algs_names = {'lgb':'LightGBM',
 'svr':'SVR',
 'rf':'RF',
 'mlp':'MLP',
 'ada':'AdaBoost',
 'resnet1d':'ResNet',
 'spectroresnet':'SpectroResNet',
 'mlpbp':'MLPBP',
 'unet1d':'UNet',
 'ppgiabp':'PPGIABP'}

In [41]:
df_SP = pd.DataFrame(columns=algs, index=[d+'_SP' for d in dbs])
df_DP = pd.DataFrame(columns=algs, index=[d+'_DP' for d in dbs])
for alg in algs:
    df_SP[alg] = df_metrics.loc[alg, 'SP_mase'].values
    df_DP[alg] = df_metrics.loc[alg, 'DP_mase'].values
df_all = pd.concat([df_SP, df_DP])

In [ ]:
df_SP.rename(columns=algs_names).to_csv('res_sp.csv')
df_DP.rename(columns=algs_names).to_csv('res_dp.csv')
df_all.rename(columns=algs_names).to_csv('res_all.csv')

In [32]:
df_DP

,lgb,svr,rf,mlp,ada,resnet1d,spectroresnet,mlpbp,unet1d,ppgiabp
sensors_DP,92.0397,90.7573,92.6325,94.0479,92.9107,100.762,102.359,99.9032,92.6204,96.6368
uci2_DP,96.0697,94.4555,96.4791,96.0463,101.392,97.064,105.311,97.9998,91.0048,94.4087
bcg_DP,99.1907,92.7542,99.7724,90.2377,101.909,98.1285,119.019,101.808,100.936,98.3687
ppgbp_DP,92.1799,90.9029,91.7618,92.7675,90.8351,93.7917,128.602,99.4124,NaN,NaN


,lgb,svr,rf,mlp,ada,resnet1d,spectroresnet,mlpbp,unet1d,ppgiabp
sensors_SP,88.7614,88.6195,90.0846,91.0273,89.4543,99.1482,103.788,100.034,88.8182,93.3954
uci2_SP,95.6026,99.0241,95.6026,103.177,95.6764,94.1217,112.778,99.6879,96.9246,96.7941
bcg_SP,98.8048,93.0726,104.724,105.505,92.8368,99.1951,155.5,100.748,99.9756,94.7638
ppgbp_SP,79.7619,80.293,80.4212,81.6911,80.7204,81.5778,115.183,100.684,NaN,NaN
sensors_DP,92.0397,90.7573,92.6325,94.0479,92.9107,100.762,102.359,99.9032,92.6204,96.6368
uci2_DP,96.0697,94.4555,96.4791,96.0463,101.392,97.064,105.311,97.9998,91.0048,94.4087
bcg_DP,99.1907,92.7542,99.7724,90.2377,101.909,98.1285,119.019,101.808,100.936,98.3687
ppgbp_DP,92.1799,90.9029,91.7618,92.7675,90.8351,93.7917,128.602,99.4124,NaN,NaN
